<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/simple_RNN_with_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch 
  print(f"You are on CoLaB with torch version: {torch.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n>>>>please load your drive...")
  COLAB = False
#assigning the gpu device if available:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
def time_fmt(t:float = 124.918)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
print(f">>>time testing: please wait\t...\n>>>time elapse:\t{time_fmt()}")


Mounted at /content/drive
You are on CoLaB with torch version: 1.8.1+cu101
>>>time testing: please wait	...
>>>time elapse:	0: 02: 04.00


In [23]:
#In this notebook we are going to use MNIST images as sequential data to train a simple RNN
#MNIST images has the shape of 1, 28,28 and in this manner we are going to treat 28 pixels as time steps(sequence length)
#and the next 28 pixels are treated as feature size for each image: We will squeeze the image to ignore the channel dimension 

In [24]:
#Importing all necessary packages:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
import time, datetime,os, sys
import numpy as np



In [25]:
#Hyperparameters for our model:
batch_size = 64
hidden_size = 256
num_layers = 2
num_classes = 10
epochs = 10
inputs_size = 28
sequence_length = 28
learning_rate = 0.001


In [26]:
#We now build our simpe rnn using the following class:

In [27]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        # Forward propagate LSTM
        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)

        # Decode the hidden state of the last time step
        out = self.fc(out)
        return out

In [28]:
model = RNN(inputs_size, hidden_size,num_layers,num_classes).to(device = device)

In [29]:
#Importing the data from torch:
train_data = datasets.MNIST(root = 'traindata_MNIST/', train = True, download = True,transform = transforms.ToTensor())
test_dta = datasets.MNIST(root = 'testdata_MNIST/', train = False, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_dta, batch_size = batch_size, shuffle = True)
train_batch_x,train_batch_y = next(iter(train_loader))
print(f"train_batch_x_shape: {train_batch_x.shape}\ttrain_batch_y_shape: {train_batch_y.shape}")

train_batch_x_shape: torch.Size([64, 1, 28, 28])	train_batch_y_shape: torch.Size([64])


In [30]:
#Getting the loss object and an optimizer:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [31]:
#The training loop:

In [34]:
tic = time.time()
for epoch in range(epochs):
  print(f"training bigin at epoch: {epoch + 1}\tplease wait...")
  for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
    #save data to device
    data = data.to(device=device).squeeze(1)
    target = target.to(device = device)
    #forward pass
    scores = model(data)
    loss = criterion(scores, target)
    #backward pass
    optimizer.zero_grad()
    loss.backward()
    #gradient descent with Adam()
    optimizer.step()

#Monitoring accuracy for training and validation:
def _model_checking(loader, model):
  if loader.dataset.train:
    print("Checking the accuracy on the training set")
  else:
    print('Checking the accuracy for validation set')
  num_correct = 0
  num_examples = 0
  model.eval()
  #No need to computing the gradients here
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device = device).squeeze(1)
      y = y.to(device = device)
      scores = model(x)
      _, predictions = scores.max(1)
      num_correct+=(predictions == y).sum()
      num_examples+=predictions.size(0)
  model.train()
  return num_correct/num_examples

print(f"training accuracy is: {float(_model_checking(train_loader, model))*100:.2f}")
print(f"validation accuracy is: {float(_model_checking(test_loader, model))*100:.2f}")
toc = time.time()
print(f"\n>>>total training and evaluation time is:\t{time_fmt(toc - tic)}")



  1%|          | 9/938 [00:00<00:10, 85.34it/s]

training bigin at epoch: 1	please wait...


  1%|          | 8/938 [00:00<00:11, 78.03it/s]

training bigin at epoch: 2	please wait...


  1%|          | 9/938 [00:00<00:10, 85.23it/s]

...

training bigin at epoch: 9	please wait...


  1%|          | 8/938 [00:00<00:11, 79.20it/s]

training bigin at epoch: 10	please wait...


100%|██████████| 938/938 [00:11<00:00, 81.94it/s]


Checking the accuracy on the training set
training accuracy is: 99.04
Checking the accuracy for validation set
validation accuracy is: 98.15

>>>total training and evaluation time is:	0: 02: 02.00
